In [45]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import mysql.connector
import re

In [17]:
column_names = ['title','url','date']
new_df = pd.DataFrame(columns = column_names)

In [18]:
idx = 0

In [57]:
chromedriver_path = 'C:/Users/kkksk/.wdm/drivers/chromedriver/win64/122.0.6261.129/chromedriver.exe'
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service)

In [58]:
cur_month = 1
cur_year = 1995
target_year = 2009

while cur_year <= target_year:
    
    url = f"https://www.google.com/search?q=crude+oil&tbs=bkt:s,cdr:1,cd_min:{cur_month}/1/{cur_year},cd_max:{cur_month+5}/31/{cur_year}&tbm=bks"
    driver.get(url)
    
    time.sleep(30)
    
    while True:
        news_results = driver.find_elements(By.CSS_SELECTOR, 'div#rso>div')
        if not news_results: 
            break  
            
        for news_div in news_results:
            try:
                Link = news_div.find_element(By.TAG_NAME, 'a').get_attribute('href')
                title_element = news_div.find_element(By.CSS_SELECTOR, 'a>h3')
                Title = title_element.text.strip(".")
                Title = re.sub(r"- \d+페이지", "", Title)
                news_elements = news_div.find_elements(By.CSS_SELECTOR, 'div')
                Date = ''  # Default Date value if not found
                if len(news_elements) >= 10:
                    info = news_elements[10].text  # Assuming 'info' is in the 11th element
                    info_text = info.split("·")
                    if len(info_text) > 1:  # Check if Date is available
                        Date = info_text[1].strip()
                
                new_df.loc[idx] = [Title, Link, Date]
                idx += 1

            except Exception as e:
                print(e)
            
        try:
            next_button = driver.find_element(By.ID, 'pnnext')
            next_button.click()
            time.sleep(1)
        except NoSuchElementException:
            break

    if cur_month == 7:  
        cur_month = 1
        cur_year += 1
    else: 
        cur_month += 6

In [59]:
new_df

,title,url,date
0,Using Microbes To Increase Recovery Of Oil. By...,https://books.google.co.kr/books?id=KgROAAAAIB...,1986. 1. 14.
1,Speculation Follows Drop In Oil Prices.,https://books.google.co.kr/books?id=WbdjAAAAIB...,1986. 1. 21.
2,Oil Price Decline Could Continue. By Kathleen ...,https://books.google.co.kr/books?id=6J4_AAAAIB...,1986. 1. 21.
3,Iran To Cut Crude Oil Output In Half In Respon...,https://books.google.co.kr/books?id=WklPAAAAIB...,1986. 1. 27.
4,Iran To Cut Crude Oil Output In Half In Respon...,https://books.google.co.kr/books?id=WUlPAAAAIB...,1986. 1. 26.
...,...,...,...
11924,There's Natural Oil. Underfoot,https://books.google.co.kr/books?id=CdwyAAAAIB...,2009. 5. 11.
11925,"Gasoline Expected To Rise, But Remain Cheap.",https://books.google.co.kr/books?id=wM80AAAAIB...,2009. 4. 15.
11926,Gas Prices.,https://books.google.co.kr/books?id=fYtcAAAAIB...,2009. 5. 22.
11927,Albany Area Gas Prices Start To Creep Back Up .,https://books.google.co.kr/books?id=YzpEAAAAIB...,2009. 2. 10.


In [60]:
new_df.to_parquet('books_additional.parquet', engine = 'pyarrow', compression = 'gzip')